In [21]:
from src.utils.coordinate_system import Dimensions
import numpy as np
from src.utils.ROI import ROI

img_dims = Dimensions(2048, 1536)

ROI.N_HORIZONTAL = 3
ROI.N_VERTICAL = 2

ROI.WIDTH = img_dims[0] / ROI.N_HORIZONTAL
ROI.HEIGHT = img_dims[1] / ROI.N_VERTICAL

rois = np.zeros(shape=(ROI.N_HORIZONTAL, ROI.N_VERTICAL), dtype=ROI)
for x in range(ROI.N_HORIZONTAL):
    for y in range(ROI.N_VERTICAL):
        rois[x, y] = ROI(x, y)

# filtered_rois = rois.flatten()[[0, 1, 2, 5]]
filtered_rois = rois.flatten()
filtered_rois

array([ROI(0;0), ROI(0;1), ROI(1;0), ROI(1;1), ROI(2;0), ROI(2;1)],
      dtype=object)

In [29]:
from scipy.spatial import distance


def roi_spatial_distance(roi1: ROI, roi2: ROI):
    """This is our adjacency measure. If the ROIs are too far apart, they can't be from the same cell, and we needn't
    compare them"""
    # get the upper left corners of both ROIs
    ul_1, _ = roi1.coordinates()
    ul_2, _ = roi2.coordinates()
    
    # Compare the upper left corners. this should be the same distance as if we compared the centers
    return  distance.euclidean((ul_1.x, ul_1.y), (ul_2.x, ul_2.y))

distance_threshold_rois = 1.5
dist_thresh_pixels = 100

In [33]:
comparisons = {}
rois_copy = list(filtered_rois)

# for each roi, check if each remaining roi in the list is within its range. 
for roi1 in filtered_rois:
    # delete this roi from rois_copy, so that comparisons are only made in one order of the pairs
    rois_copy.remove(roi1)

    # get the upper left corner for roi1
    ul1, _ = roi1.coordinates()

    # check which rois are within the distance for this roi
    within_dist = []
    for roi2 in rois_copy:
        ul2, _ = roi2.coordinates()
        print(roi1, roi2, ' : ', f'{roi_spatial_distance(roi1, roi2):.2f}', ':',
              roi_spatial_distance(roi1, roi2) < distance_threshold_rois)
        if roi_spatial_distance(roi1, roi2) < distance_threshold_rois:
            within_dist.append(roi2)

    comparisons[roi1] = within_dist

comparisons

(0;0) (0;1)  :  1.00 : True
(0;0) (1;0)  :  1.00 : True
(0;0) (1;1)  :  1.41 : True
(0;0) (2;0)  :  2.00 : False
(0;0) (2;1)  :  2.24 : False
(0;1) (1;0)  :  1.41 : True
(0;1) (1;1)  :  1.00 : True
(0;1) (2;0)  :  2.24 : False
(0;1) (2;1)  :  2.00 : False
(1;0) (1;1)  :  1.00 : True
(1;0) (2;0)  :  1.00 : True
(1;0) (2;1)  :  1.41 : True
(1;1) (2;0)  :  1.41 : True
(1;1) (2;1)  :  1.00 : True
(2;0) (2;1)  :  1.00 : True


{ROI(0;0): [ROI(0;1), ROI(1;0), ROI(1;1)],
 ROI(0;1): [ROI(1;0), ROI(1;1)],
 ROI(1;0): [ROI(1;1), ROI(2;0), ROI(2;1)],
 ROI(1;1): [ROI(2;0), ROI(2;1)],
 ROI(2;0): [ROI(2;1)],
 ROI(2;1): []}